1Round Propose

In [ ]:
import os
import base64
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
import re

# OpenAI API Key (replace with your own API key)
my_api_key = "your_openai_api_key"
client = OpenAI(api_key=my_api_key, timeout=20.0)

class DialogModel:
    def __init__(self, engine="gpt-4o"):
        """
        Initialize the DialogModel class.
        :param engine: Name of the GPT engine to use
        """
        self.engine = engine
        self.messages = []

    def set_prompts(self, system_prompt, user_prompt):
        """
        Set the system prompt and user prompt and initialize the message array.
        :param system_prompt: Common system prompt
        :param user_prompt: User prompt specific to the model
        """
        self.messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
    def send_message(self, message=None):
        """
        Send a message to the model and receive the response.
        :param message: Message to send
        :return: Model response
        """
        if message:
            self.messages.append({"role": "user", "content": message})
        
        for attempt in range(5):  # Maximum 5 attempts
            try:
                response = client.chat.completions.create(
                    model=self.engine,
                    messages=self.messages,
                    max_tokens=300,
                    temperature=1,
                )
                break
            except Exception as e:
                print(f"An error occurred during send_message attempt {attempt + 1}: {e}")
                if 'Connection error' in str(e):
                    print("Connection error detected. Waiting for 5 minutes before retrying...")
                    time.sleep(300)  # Wait for 5 minutes
                else:
                    time.sleep(1)  # Wait for 1 second for other errors
                if attempt == 4:
                    print(f"Failed to process message after 5 attempts. Skipping file.")
                    return None

        reply = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": reply})
        return reply

    def reset_dialog(self):
        """
        Reset the dialog history.
        """
        self.messages = []
        
    def update_message_with_reply(self, reply):
        """
        Add the other model's response to the current model's message array.
        :param reply: Response message to add
        """
        self.messages.append({"role": "user", "content": reply})
    
    def add_round_marker(self, round_number):
        """
        Add a round marker to the conversation.
        :param round_number: Round number
        """
        round_marker = f"===round{round_number}==="
        self.messages.append({"role": "user", "content": round_marker})

def extract_proposal(dialogue):
    for attempt in range(5):  # Maximum 5 attempts
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Based on the following dialogue, extract the amount the proposer proposed for him or her self against the responder. Answer with only one number. Do not use '$'."},
                    {"role": "user", "content": f"Dialogue:\n{dialogue}"}
                ],
                temperature=0.0,
                max_tokens=150
            )
            break
        except Exception as e:
            print(f"An error occurred during extract_proposal attempt {attempt + 1}: {e}")
            if 'Connection error' in str(e):
                print("Connection error detected. Waiting for 5 minutes before retrying...")
                time.sleep(300)  # Wait for 5 minutes
            else:
                time.sleep(1)  # Wait for 1 second for other errors
            if attempt == 4:
                print(f"Failed to extract proposal after 5 attempts. Skipping this step.")
                return None

    return response.choices[0].message.content.strip()

def extract_acceptance(dialogue):
    for attempt in range(5):  # Maximum 5 attempts
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Based on the following dialogue, If the responder accepts the proposal, print '1', and if it rejects, print '0'. Answer with only one number."},
                    {"role": "user", "content": f"Dialogue:\n{dialogue}"}
                ],
                temperature=0.0,
                max_tokens=150
            )
            break
        except Exception as e:
            print(f"An error occurred during extract_acceptance attempt {attempt + 1}: {e}")
            if 'Connection error' in str(e):
                print("Connection error detected. Waiting for 5 minutes before retrying...")
                time.sleep(300)  # Wait for 5 minutes
            else:
                time.sleep(1)  # Wait for 1 second for other errors
            if attempt == 4:
                print(f"Failed to extract acceptance after 5 attempts. Skipping this step.")
                return None

    return response.choices[0].message.content.strip()

def automated_dialogue(model1, manual_responses, number_of_turns):
    """
    Conducts a conversation between Model 1 and manual responses.
    :param model1: Model to generate dialogue
    :param manual_responses: Manual responses for the other model
    :param number_of_turns: Number of dialogue turns
    """
    dialogue_results = ""  # To store the entire dialogue
    rounds_data = []  # To store data for each round
    dialogue_data = []

    for turn in range(number_of_turns):
        model1.add_round_marker(turn + 1)
        round_info = {'round': turn + 1, 'model1_role': '', 'model1_text': '', 'proposal_amount': '',
                      'model2_text': '', 'proposal_acceptance': ''}

        if turn % 2 == 0:
            reply_from_model1 = model1.send_message()
            if reply_from_model1 is None:
                continue
            proposal_amount = extract_proposal(reply_from_model1)
            round_info['model1_role'] = 'Proposer'
            round_info['model1_text'] = reply_from_model1
            round_info['proposal_amount'] = proposal_amount
            dialogue_results += f"Round {turn+1}, Model 1 (Proposer): {reply_from_model1}\n"
            model1.update_message_with_reply(manual_responses[turn])
            proposal_acceptance = extract_acceptance(manual_responses[turn])
            round_info['model2_text'] = manual_responses[turn]
            round_info['proposal_acceptance'] = proposal_acceptance
            dialogue_results += f"Round {turn+1}, Model 2 (Responder): {manual_responses[turn]}\n"
            dialogue_data.extend([round_info['model1_text'], round_info['model2_text']])
        else:
            model1.update_message_with_reply(manual_responses[turn])
            proposal_amount = extract_proposal(manual_responses[turn])
            round_info['model2_text'] = manual_responses[turn]
            round_info['proposal_amount'] = proposal_amount
            dialogue_results += f"Round {turn+1}, Model 2 (Proposer): {manual_responses[turn]}\n"
            reply_from_model1 = model1.send_message()
            if reply_from_model1 is None:
                continue
            proposal_acceptance = extract_acceptance(reply_from_model1)
            round_info['model1_role'] = 'Responder'
            round_info['model1_text'] = reply_from_model1
            round_info['proposal_acceptance'] = proposal_acceptance
            dialogue_results += f"Round {turn+1}, Model 1 (Responder): {reply_from_model1}\n"
            dialogue_data.extend([round_info['model2_text'], round_info['model1_text']])

        rounds_data.append(round_info)
        dialogue_data.extend([round_info['proposal_amount'], round_info['proposal_acceptance']])

    columns = []
    for i in range(number_of_turns):
        columns.extend([
            f"text_{i*2+1}", f"text_{i*2+2}",
            f"proposal_amount_round_{i+1}", f"proposal_acceptance_round_{i+1}"
        ])
    df = pd.DataFrame([dialogue_data], columns=columns)
    return dialogue_results, rounds_data, df

def encode_image(image_path):
    """
    Encode an image to base64 format.
    :param image_path: Path to the image
    :return: Base64 encoded image
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def save_progress(df_all: pd.DataFrame, base_path: str, start_file_num: int, end_file_num: int):
    """
    Save progress to an Excel file.
    :param df_all: The complete DataFrame
    :param base_path: Path to save the file
    :param start_file_num: Starting file number
    :param end_file_num: Ending file number
    """
    columns_reordered = ['image_file']
    number_of_turns = 4  # Assuming a fixed number of turns
    columns_text = []
    columns_proposals = []
    columns_acceptance = []
    for i in range(1, number_of_turns + 1):
        columns_text.extend([f"text_{i*2-1}", f"text_{i*2}"])
        columns_proposals.append(f"proposal_amount_round_{i}")
        columns_acceptance.append(f"proposal_acceptance_round_{i}")

    columns_reordered.extend(columns_text + columns_proposals + columns_acceptance)
    df_all = df_all[columns_reordered]

    # Save DataFrame to Excel file
    excel_file_path = os.path.join(base_path, f'study1_gpt_1round_{start_file_num}-{end_file_num}.xlsx')
    df_all.to_excel(excel_file_path, index=False)
    print(f"DataFrame saved to {excel_file_path}")

# Example execution
manual_responses = [
    "As the responder, I accept your proposal.",
    "As the proposer, I suggest an even split, $50 for me and 50$ for you.",
    "As the responder, I accept your proposal.",
    "As the proposer, I suggest a division of $75 for me and $25 for you."
]

system_prompt1 = "In the following conversation, you are one of the two players participating in a negotiation game."

base_path = "/path/to/your/folder"

# User input for file processing range
start_file_num = int(input("Enter the starting file number (numeric only): "))
num_files_to_process = int(input("Enter the number of files to process: "))
end_file_num = start_file_num + num_files_to_process - 1

# Function to extract number for sorting files numerically
def extract_number(file_name):
    match = re.search(r'(\d+)', file_name)
    return int(match.group(1)) if match else -1

files = sorted([file for file in os.listdir(base_path) if file.endswith('.png') and int(file.split('.')[0]) >= start_file_num], key=extract_number)[:num_files_to_process]
df_all = pd.DataFrame()
skipped_files = []
print(f"Processing files from {start_file_num} to {end_file_num} (total {num_files_to_process} files).")

try:
    # Use tqdm to show progress
    for file in tqdm(files, desc="Processing files"):
        image_num = int(file.split('.')[0])
        image_path = os.path.join(base_path, file)
        base64_image = encode_image(image_path)
        user_prompt_model1 = [
            {"type": "text", "text": "You are the character in the following image:"},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}},
            {"type": "text", "text": """
            Consider your character's traits in upcoming conversations.

            Let's play an ultimatum game. The rules of the game are as follows: 
            (1) Two players divide $100 in each round. 
            (2) In each round, one player proposes a division ratio, and the other player can only accept or reject the proposal; one can not make a counter-proposal. 
            (3) If the proposal is accepted, the money is divided according to the proposed ratio; if rejected, neither player gets any money. 
            (4) The game consists of a total of four rounds, with players alternating roles between proposing and responding in each round. 
            
            You are the proposer in the first and third round.
            
            Do not declare '===round#===' on your own.

            The proposer should wait the '===round#===' notice and then start making the proposal.
            """}
        ]
        model1 = DialogModel()
        model1.set_prompts(system_prompt1, user_prompt_model1)
        number_of_turns = 4
        result, rounds_results, df = automated_dialogue(model1, manual_responses, number_of_turns)
        if result:
            df['image_file'] = image_num  # Add image file name to DataFrame (numeric only)
            df_all = pd.concat([df_all, df], ignore_index=True)
        else:
            skipped_files.append(file)
        model1.reset_dialog()
except Exception as e:
    print(f"An error occurred: {e}")
    save_progress(df_all, base_path, start_file_num, image_num)
    print(f"Skipped files due to errors: {', '.join(skipped_files)}")
finally:
    save_progress(df_all, base_path, start_file_num, end_file_num)
    if skipped_files:
        print(f"Skipped files: {', '.join(skipped_files)}")


2Round Propose

In [ ]:
import os
import base64
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
import re

# OpenAI API Key (replace with your own API key)
my_api_key = "your_openai_api_key"
client = OpenAI(api_key=my_api_key, timeout=20.0)

class DialogModel:
    def __init__(self, engine="gpt-4o"):
        """
        Initialize the DialogModel class.
        :param engine: Name of the GPT engine to use
        """
        self.engine = engine
        self.messages = []

    def set_prompts(self, system_prompt, user_prompt):
        """
        Set the system prompt and user prompt and initialize the message array.
        :param system_prompt: Common system prompt
        :param user_prompt: User prompt specific to the model
        """
        self.messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
    def send_message(self, message=None):
        """
        Send a message to the model and receive the response.
        :param message: Message to send
        :return: Model response
        """
        if message:
            self.messages.append({"role": "user", "content": message})
        
        for attempt in range(5):  # Maximum 5 attempts
            try:
                response = client.chat.completions.create(
                    model=self.engine,
                    messages=self.messages,
                    max_tokens=300,
                    temperature=1,
                )
                break
            except Exception as e:
                print(f"An error occurred during send_message attempt {attempt + 1}: {e}")
                if 'Connection error' in str(e):
                    print("Connection error detected. Waiting for 5 minutes before retrying...")
                    time.sleep(300)  # Wait for 5 minutes
                else:
                    time.sleep(1)  # Wait for 1 second for other errors
                if attempt == 4:
                    print(f"Failed to process message after 5 attempts. Skipping file.")
                    return None

        reply = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": reply})
        return reply

    def reset_dialog(self):
        """
        Reset the dialog history.
        """
        self.messages = []
        
    def update_message_with_reply(self, reply):
        """
        Add the other model's response to the current model's message array.
        :param reply: Response message to add
        """
        self.messages.append({"role": "user", "content": reply})
    
    def add_round_marker(self, round_number):
        """
        Add a round marker to the conversation.
        :param round_number: Round number
        """
        round_marker = f"===round{round_number}==="
        self.messages.append({"role": "user", "content": round_marker})

def extract_proposal(dialogue):
    for attempt in range(5):  # Maximum 5 attempts
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Based on the following dialogue, extract the amount the proposer proposed for him or her self against the responder. Answer with only one number. Do not use '$'."},
                    {"role": "user", "content": f"Dialogue:\n{dialogue}"}
                ],
                temperature=0.0,
                max_tokens=150
            )
            break
        except Exception as e:
            print(f"An error occurred during extract_proposal attempt {attempt + 1}: {e}")
            if 'Connection error' in str(e):
                print("Connection error detected. Waiting for 5 minutes before retrying...")
                time.sleep(300)  # Wait for 5 minutes
            else:
                time.sleep(1)  # Wait for 1 second for other errors
            if attempt == 4:
                print(f"Failed to extract proposal after 5 attempts. Skipping this step.")
                return None

    return response.choices[0].message.content.strip()

def extract_acceptance(dialogue):
    for attempt in range(5):  # Maximum 5 attempts
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Based on the following dialogue, If the responder accepts the proposal, print '1', and if it rejects, print '0'. Answer with only one number."},
                    {"role": "user", "content": f"Dialogue:\n{dialogue}"}
                ],
                temperature=0.0,
                max_tokens=150
            )
            break
        except Exception as e:
            print(f"An error occurred during extract_acceptance attempt {attempt + 1}: {e}")
            if 'Connection error' in str(e):
                print("Connection error detected. Waiting for 5 minutes before retrying...")
                time.sleep(300)  # Wait for 5 minutes
            else:
                time.sleep(1)  # Wait for 1 second for other errors
            if attempt == 4:
                print(f"Failed to extract acceptance after 5 attempts. Skipping this step.")
                return None

    return response.choices[0].message.content.strip()

def automated_dialogue(model1, manual_responses, number_of_turns):
    """
    Conducts a conversation between Model 1 and manual responses.
    :param model1: Model to generate dialogue
    :param manual_responses: Manual responses for the other model
    :param number_of_turns: Number of dialogue turns
    """
    dialogue_results = ""  # To store the entire dialogue
    rounds_data = []  # To store data for each round
    dialogue_data = []

    for turn in range(number_of_turns):
        model1.add_round_marker(turn + 1)
        round_info = {'round': turn + 1, 'model1_role': '', 'model1_text': '', 'proposal_amount': '',
                      'model2_text': '', 'proposal_acceptance': ''}

        if turn % 2 == 0:
            # Model 2 (manual_responses) is the proposer in even rounds
            model1.update_message_with_reply(manual_responses[turn])
            proposal_amount = extract_proposal(manual_responses[turn])
            round_info['model2_text'] = manual_responses[turn]
            round_info['proposal_amount'] = proposal_amount
            dialogue_results += f"Round {turn+1}, Model 2 (Proposer): {manual_responses[turn]}\n"
            reply_from_model1 = model1.send_message()
            if reply_from_model1 is None:
                continue
            proposal_acceptance = extract_acceptance(reply_from_model1)
            round_info['model1_role'] = 'Responder'
            round_info['model1_text'] = reply_from_model1
            round_info['proposal_acceptance'] = proposal_acceptance
            dialogue_results += f"Round {turn+1}, Model 1 (Responder): {reply_from_model1}\n"
            dialogue_data.extend([round_info['model2_text'], round_info['model1_text']])
        else:
            # Model 1 is the proposer in odd rounds
            reply_from_model1 = model1.send_message()
            if reply_from_model1 is None:
                continue
            proposal_amount = extract_proposal(reply_from_model1)
            round_info['model1_role'] = 'Proposer'
            round_info['model1_text'] = reply_from_model1
            round_info['proposal_amount'] = proposal_amount
            dialogue_results += f"Round {turn+1}, Model 1 (Proposer): {reply_from_model1}\n"
            model1.update_message_with_reply(manual_responses[turn])
            proposal_acceptance = extract_acceptance(manual_responses[turn])
            round_info['model2_text'] = manual_responses[turn]
            round_info['proposal_acceptance'] = proposal_acceptance
            dialogue_results += f"Round {turn+1}, Model 2 (Responder): {manual_responses[turn]}\n"
            dialogue_data.extend([round_info['model1_text'], round_info['model2_text']])

        rounds_data.append(round_info)
        dialogue_data.extend([round_info['proposal_amount'], round_info['proposal_acceptance']])

    columns = []
    for i in range(number_of_turns):
        columns.extend([
            f"text_{i*2+1}", f"text_{i*2+2}",
            f"proposal_amount_round_{i+1}", f"proposal_acceptance_round_{i+1}"
        ])
    df = pd.DataFrame([dialogue_data], columns=columns)
    return dialogue_results, rounds_data, df

def encode_image(image_path):
    """
    Encode an image to base64 format.
    :param image_path: Path to the image
    :return: Base64 encoded image
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def save_progress(df_all: pd.DataFrame, base_path: str, start_file_num: int, end_file_num: int):
    """
    Save progress to an Excel file.
    :param df_all: The complete DataFrame
    :param base_path: Path to save the file
    :param start_file_num: Starting file number
    :param end_file_num: Ending file number
    """
    columns_reordered = ['image_file']
    number_of_turns = 4  # Assuming a fixed number of turns
    columns_text = []
    columns_proposals = []
    columns_acceptance = []
    for i in range(1, number_of_turns + 1):
        columns_text.extend([f"text_{i*2-1}", f"text_{i*2}"])
        columns_proposals.append(f"proposal_amount_round_{i}")
        columns_acceptance.append(f"proposal_acceptance_round_{i}")

    columns_reordered.extend(columns_text + columns_proposals + columns_acceptance)
    df_all = df_all[columns_reordered]

    # Save DataFrame to Excel file
    excel_file_path = os.path.join(base_path, f'study1_gpt_2round_{start_file_num}-{end_file_num}.xlsx')
    df_all.to_excel(excel_file_path, index=False)
    print(f"DataFrame saved to {excel_file_path}")

# Example execution
manual_responses = [
    "As the proposer, I suggest an even split, $50 for me and $50 for you.",
    "As the responder, I accept your proposal.",
    "As the proposer, I suggest a division of $75 for me and $25 for you.",
    "As the responder, I accept your proposal.",
]

system_prompt1 = "In the following conversation, you are one of the two players participating in a negotiation game."

base_path = "/path/to/your/folder"
# User input for file processing range
start_file_num = int(input("Enter the starting file number (numeric only): "))
num_files_to_process = int(input("Enter the number of files to process: "))
end_file_num = start_file_num + num_files_to_process - 1

# Function to extract number for sorting files numerically
def extract_number(file_name):
    match = re.search(r'(\d+)', file_name)
    return int(match.group(1)) if match else -1

files = sorted([file for file in os.listdir(base_path) if file.endswith('.png') and int(file.split('.')[0]) >= start_file_num], key=extract_number)[:num_files_to_process]
df_all = pd.DataFrame()
skipped_files = []
print(f"Processing files from {start_file_num} to {end_file_num} (total {num_files_to_process} files).")

try:
    # Use tqdm to show progress
    for file in tqdm(files, desc="Processing files"):
        image_num = int(file.split('.')[0])
        image_path = os.path.join(base_path, file)
        base64_image = encode_image(image_path)
        user_prompt_model1 = [
            {"type": "text", "text": "You are the character in the following image:"},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}},
            {"type": "text", "text": """
            Consider your character's traits in upcoming conversations.

            Let's play an ultimatum game. The rules of the game are as follows: 
            (1) Two players divide $100 in each round. 
            (2) In each round, one player proposes a division ratio, and the other player can only accept or reject the proposal; one can not make a counter-proposal. 
            (3) If the proposal is accepted, the money is divided according to the proposed ratio; if rejected, neither player gets any money. 
            (4) The game consists of a total of four rounds, with players alternating roles between proposing and responding in each round. 
            
            You are the proposer in the second and fourth round.
            
            Do not declare '===round#===' on your own.

            The proposer should wait the '===round#===' notice and then start making the proposal.
            """}
        ]
        model1 = DialogModel()
        model1.set_prompts(system_prompt1, user_prompt_model1)
        number_of_turns = 4
        result, rounds_results, df = automated_dialogue(model1, manual_responses, number_of_turns)
        if result:
            df['image_file'] = image_num  # Add image file name to DataFrame (numeric only)
            df_all = pd.concat([df_all, df], ignore_index=True)
        else:
            skipped_files.append(file)
        model1.reset_dialog()
except Exception as e:
    print(f"An error occurred: {e}")
    save_progress(df_all, base_path, start_file_num, image_num)
    print(f"Skipped files due to errors: {', '.join(skipped_files)}")
finally:
    save_progress(df_all, base_path, start_file_num, end_file_num)
    if skipped_files:
        print(f"Skipped files: {', '.join(skipped_files)}")
